In [17]:
import pandas as pd

year = 2023

df_tornei = pd.read_csv(f"tornei/tornei_{str(year)[-2:]}.csv")
df_iscrizioni = pd.read_csv(f"df_iscrizioni_player_event/df_{year}.csv")

# 1) Liste distinte
players = df_iscrizioni["PlayerCode"].dropna().unique()
events  = df_tornei["EventId"].dropna().unique()

# 2) Griglia completa (tutte le combinazioni player-event)
grid = (
    pd.MultiIndex.from_product([players, events], names=["PlayerCode", "EventId"])
      .to_frame(index=False)
)

# 3) Tieni solo le colonne chiave (una riga = "iscritto")
iscr_key = df_iscrizioni[["PlayerCode", "EventId"]].drop_duplicates()
iscr_key["iscritto"] = 1

# 4) Merge per marcare iscritti / non iscritti
out = grid.merge(iscr_key, on=["PlayerCode", "EventId"], how="left")
out["iscritto"] = out["iscritto"].fillna(0).astype(int)

# 5) Aggiungi info torneo (una sola volta)
out = out.merge(df_tornei, on="EventId", how="left")

display(out)


,PlayerCode,EventId,iscritto,year,date_tournament,EventName,EventType,EventCountry,Surface,TotPrizeMoney
0,mv14,319,1,2023,2023-07-31,Kitzbuhel,250,Austria,Clay,562815
1,mv14,414,1,2023,2023-07-24,Hamburg,500,Germany,Clay,1831515
2,mv14,316,1,2023,2023-07-17,Bastad,250,Sweden,Clay,562815
3,mv14,2841,1,2023,2023-04-17,Banja Luka,250,Bosnia & Herzegovina,Clay,562815
4,mv14,807,1,2023,2023-02-27,Acapulco,500,Mexico,Hard,2013940
...,...,...,...,...,...,...,...,...,...,...
18355,m0bn,499,0,2023,2023-02-13,Delray Beach,250,"FL, U.S.A.",Hard,642735
18356,m0bn,424,0,2023,2023-02-06,Dallas,250,"TX, U.S.A.",Hard,737170
18357,m0bn,747,0,2023,2023-09-25,Beijing,500,China,Hard,3633875
18358,m0bn,311,0,2023,2023-06-19,London / Queen's Club,500,Great Britain,Grass,2195175


In [18]:
# aggiunta colonna di same nationality player-event
tornei = pd.read_csv("df_iscrizioni_player_event/df_2025.csv")
players = pd.read_csv("ranking_from_atp/atp_senza_ripetizioni_2025.csv")

#seleziona solo playercode, eventid, eventcountry
tornei = tornei[["PlayerCode", "EventId", "EventCountry"]]
players = players[["Player Id", "Nationality"]]
# df = pd.concat([tornei["PlayerCode"], tornei["EventId"], tornei["EventCountry"], players[""]])

df = tornei.merge(
    players,
    left_on="PlayerCode",
    right_on="Player Id",
    how="left"
)
# opzionale: elimina la colonna duplicata
# df = df.drop(columns=["Player Id"])

geo1 = pd.read_csv("geografia.csv")
geo2 = pd.read_csv("geografia.csv")
#df
df_primo_join = df.merge(geo1,
    left_on="EventCountry",
    right_on="name",
    how="left"
)

#nationality in upper case
df_primo_join["Nationality"] = df_primo_join["Nationality"].str.upper()


df_finale = df_primo_join[df_primo_join["Nationality"] == df_primo_join["alpha-3"]]
df_finale=df_finale[["PlayerCode", "EventId", "EventCountry","Nationality"]]
df_finale

,PlayerCode,EventId,EventCountry,Nationality
0,s0re,5014,China,CHN
1,s0re,747,China,CHN
2,s0re,7581,China,CHN
13,bd06,8994,Spain,ESP
19,bd06,1536,Spain,ESP
...,...,...,...,...
9632,b0bu,7009,France,FRA
9637,b0bu,2973,France,FRA
9639,b0bu,9162,France,FRA
9641,b0bu,7874,France,FRA


In [19]:
# add column same_n
out["Same_Nationality"] = 0

for index, row in out.iterrows():
    player_code = row["PlayerCode"]
    event_id = row["EventId"]
    
    match = df_finale[
        (df_finale["PlayerCode"] == player_code) & 
        (df_finale["EventId"] == event_id)
    ]
    
    if not match.empty:
        out.at[index, "Same_Nationality"] = 1
out

,PlayerCode,EventId,iscritto,year,date_tournament,EventName,EventType,EventCountry,Surface,TotPrizeMoney,Same_Nationality
0,mv14,319,1,2023,2023-07-31,Kitzbuhel,250,Austria,Clay,562815,0
1,mv14,414,1,2023,2023-07-24,Hamburg,500,Germany,Clay,1831515,0
2,mv14,316,1,2023,2023-07-17,Bastad,250,Sweden,Clay,562815,0
3,mv14,2841,1,2023,2023-04-17,Banja Luka,250,Bosnia & Herzegovina,Clay,562815,0
4,mv14,807,1,2023,2023-02-27,Acapulco,500,Mexico,Hard,2013940,0
...,...,...,...,...,...,...,...,...,...,...,...
18355,m0bn,499,0,2023,2023-02-13,Delray Beach,250,"FL, U.S.A.",Hard,642735,0
18356,m0bn,424,0,2023,2023-02-06,Dallas,250,"TX, U.S.A.",Hard,737170,0
18357,m0bn,747,0,2023,2023-09-25,Beijing,500,China,Hard,3633875,0
18358,m0bn,311,0,2023,2023-06-19,London / Queen's Club,500,Great Britain,Grass,2195175,0


In [20]:
out.to_csv(f"df_per_aggiungere_pts_def/df_{year}.csv", index=False)